# Reddit AITA Finetuned Model Evaluation

In [5]:
%pip install evaluate rouge_score peft

                                              0.0/199.1 kB ? eta -:--:--
     -------------------------------------- 199.1/199.1 kB 5.9 MB/s eta 0:00:00
                                              0.0/290.1 kB ? eta -:--:--
     ------------------------------------- 290.1/290.1 kB 18.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1 -> 24.0
[notice] To update, run: C:\Users\mattb\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
from huggingface_hub import login
login()